In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.keras.models import Model

import pathlib
import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [100]:
#데이터 불러오기
raw_data=pd.read_csv('full_player_comparison_final.csv')

In [101]:
raw_data

,Unnamed: 0,name,preprocessed
0,0,A. 고메스,"['D. 마라도나', '이용', '리오넬 메시', '루드 굴리트']"
1,2,A. 골로빈,"['사울', '산티 미나', '악셀 비첼', '루카 모드리치', '루드 굴리트']"
2,3,A. 과르다도,"['마르코스 로호', 'P. 비에이라', '에마뉘엘 프티', '라파엘 바란', '조..."
3,4,A. 그랑크비스트,"['라파엘 바란', '다비드 루이스', 'R. 푸네스 모리']"
4,5,A. 그리즈만,"['킬리안 음바페', '모하메드 살라', '가레스 베일', '네이마르 Jr.', '..."
...,...,...,...
1415,1799,히바우두,"['가레스 베일', '라울', '요한 크루이프', '루드 굴리트']"
1416,1800,히샤를리송,"['프랭크 램파드', '로스 바클리', '루드 굴리트', '미하엘 발락']"
1417,1801,히카르두 굴라르,"['폴 포그바', '쿠티뉴', '페드로 포로']"
1418,1802,히카르두 오르타,"['데헤아', 'P. 시세', '카시야스', '케일러 나바스', '페트르 체흐']"


In [170]:
raw_data[raw_data.name == '피지']

,Unnamed: 0,name,list,string,new_list
1339,1706,피지,"[코케, 카라 음보지, 세스크 파브레가스, 네이마르 Jr., 루드 굴리트, 피지]","코케,카라 음보지,세스크 파브레가스,네이마르 Jr.,루드 굴리트,피지","[코케, 카라 음보지, 세스크 파브레가스, 네이마르 Jr., 루드 굴리트, 피지]"


In [102]:
def to_list(x):
    return [name.strip()[1:-1] for name in x[1:-1].split(',')]

In [103]:
raw_data['list'] = raw_data['preprocessed'].apply(lambda x: to_list(x))

In [104]:
del raw_data['preprocessed']

In [105]:
raw_data

,Unnamed: 0,name,list
0,0,A. 고메스,"[D. 마라도나, 이용, 리오넬 메시, 루드 굴리트]"
1,2,A. 골로빈,"[사울, 산티 미나, 악셀 비첼, 루카 모드리치, 루드 굴리트]"
2,3,A. 과르다도,"[마르코스 로호, P. 비에이라, 에마뉘엘 프티, 라파엘 바란, 조던 모리스]"
3,4,A. 그랑크비스트,"[라파엘 바란, 다비드 루이스, R. 푸네스 모리]"
4,5,A. 그리즈만,"[킬리안 음바페, 모하메드 살라, 가레스 베일, 네이마르 Jr., 리오넬 메시]"
...,...,...,...
1415,1799,히바우두,"[가레스 베일, 라울, 요한 크루이프, 루드 굴리트]"
1416,1800,히샤를리송,"[프랭크 램파드, 로스 바클리, 루드 굴리트, 미하엘 발락]"
1417,1801,히카르두 굴라르,"[폴 포그바, 쿠티뉴, 페드로 포로]"
1418,1802,히카르두 오르타,"[데헤아, P. 시세, 카시야스, 케일러 나바스, 페트르 체흐]"


# 단어에 숫자 부여하기

In [106]:
raw_data['string'] = [','.join(map(str, row[2])) +','+row[1] for row in raw_data.values ]
raw_data

,Unnamed: 0,name,list,string
0,0,A. 고메스,"[D. 마라도나, 이용, 리오넬 메시, 루드 굴리트]","D. 마라도나,이용,리오넬 메시,루드 굴리트,A. 고메스"
1,2,A. 골로빈,"[사울, 산티 미나, 악셀 비첼, 루카 모드리치, 루드 굴리트]","사울,산티 미나,악셀 비첼,루카 모드리치,루드 굴리트,A. 골로빈"
2,3,A. 과르다도,"[마르코스 로호, P. 비에이라, 에마뉘엘 프티, 라파엘 바란, 조던 모리스]","마르코스 로호,P. 비에이라,에마뉘엘 프티,라파엘 바란,조던 모리스,A. 과르다도"
3,4,A. 그랑크비스트,"[라파엘 바란, 다비드 루이스, R. 푸네스 모리]","라파엘 바란,다비드 루이스,R. 푸네스 모리,A. 그랑크비스트"
4,5,A. 그리즈만,"[킬리안 음바페, 모하메드 살라, 가레스 베일, 네이마르 Jr., 리오넬 메시]","킬리안 음바페,모하메드 살라,가레스 베일,네이마르 Jr.,리오넬 메시,A. 그리즈만"
...,...,...,...,...
1415,1799,히바우두,"[가레스 베일, 라울, 요한 크루이프, 루드 굴리트]","가레스 베일,라울,요한 크루이프,루드 굴리트,히바우두"
1416,1800,히샤를리송,"[프랭크 램파드, 로스 바클리, 루드 굴리트, 미하엘 발락]","프랭크 램파드,로스 바클리,루드 굴리트,미하엘 발락,히샤를리송"
1417,1801,히카르두 굴라르,"[폴 포그바, 쿠티뉴, 페드로 포로]","폴 포그바,쿠티뉴,페드로 포로,히카르두 굴라르"
1418,1802,히카르두 오르타,"[데헤아, P. 시세, 카시야스, 케일러 나바스, 페트르 체흐]","데헤아,P. 시세,카시야스,케일러 나바스,페트르 체흐,히카르두 오르타"


In [107]:
my_str= raw_data['string'].str.cat(sep=', ')
words = my_str.split(",")
#빈칸지우기
words = [x.strip(' ') for x in words]

In [108]:
len(words)

6714

In [109]:
#빈칸지우고 유일한것만 남기기
words= list(filter(None, words))
words= list(set(words))

In [110]:
#단어에 숫자 부여하기
word2int = {}
for i,word in enumerate(words):
    word2int[word] = i

In [111]:
len(word2int)

1481

In [112]:
def appending(name,text):
    text.append(name)
    return text

In [113]:
raw_data['new_list'] = raw_data.apply(lambda row: appending(row['name'],row['list']), axis=1)

In [114]:
raw_data

,Unnamed: 0,name,list,string,new_list
0,0,A. 고메스,"[D. 마라도나, 이용, 리오넬 메시, 루드 굴리트, A. 고메스]","D. 마라도나,이용,리오넬 메시,루드 굴리트,A. 고메스","[D. 마라도나, 이용, 리오넬 메시, 루드 굴리트, A. 고메스]"
1,2,A. 골로빈,"[사울, 산티 미나, 악셀 비첼, 루카 모드리치, 루드 굴리트, A. 골로빈]","사울,산티 미나,악셀 비첼,루카 모드리치,루드 굴리트,A. 골로빈","[사울, 산티 미나, 악셀 비첼, 루카 모드리치, 루드 굴리트, A. 골로빈]"
2,3,A. 과르다도,"[마르코스 로호, P. 비에이라, 에마뉘엘 프티, 라파엘 바란, 조던 모리스, A....","마르코스 로호,P. 비에이라,에마뉘엘 프티,라파엘 바란,조던 모리스,A. 과르다도","[마르코스 로호, P. 비에이라, 에마뉘엘 프티, 라파엘 바란, 조던 모리스, A...."
3,4,A. 그랑크비스트,"[라파엘 바란, 다비드 루이스, R. 푸네스 모리, A. 그랑크비스트]","라파엘 바란,다비드 루이스,R. 푸네스 모리,A. 그랑크비스트","[라파엘 바란, 다비드 루이스, R. 푸네스 모리, A. 그랑크비스트]"
4,5,A. 그리즈만,"[킬리안 음바페, 모하메드 살라, 가레스 베일, 네이마르 Jr., 리오넬 메시, A...","킬리안 음바페,모하메드 살라,가레스 베일,네이마르 Jr.,리오넬 메시,A. 그리즈만","[킬리안 음바페, 모하메드 살라, 가레스 베일, 네이마르 Jr., 리오넬 메시, A..."
...,...,...,...,...,...
1415,1799,히바우두,"[가레스 베일, 라울, 요한 크루이프, 루드 굴리트, 히바우두]","가레스 베일,라울,요한 크루이프,루드 굴리트,히바우두","[가레스 베일, 라울, 요한 크루이프, 루드 굴리트, 히바우두]"
1416,1800,히샤를리송,"[프랭크 램파드, 로스 바클리, 루드 굴리트, 미하엘 발락, 히샤를리송]","프랭크 램파드,로스 바클리,루드 굴리트,미하엘 발락,히샤를리송","[프랭크 램파드, 로스 바클리, 루드 굴리트, 미하엘 발락, 히샤를리송]"
1417,1801,히카르두 굴라르,"[폴 포그바, 쿠티뉴, 페드로 포로, 히카르두 굴라르]","폴 포그바,쿠티뉴,페드로 포로,히카르두 굴라르","[폴 포그바, 쿠티뉴, 페드로 포로, 히카르두 굴라르]"
1418,1802,히카르두 오르타,"[데헤아, P. 시세, 카시야스, 케일러 나바스, 페트르 체흐, 히카르두 오르타]","데헤아,P. 시세,카시야스,케일러 나바스,페트르 체흐,히카르두 오르타","[데헤아, P. 시세, 카시야스, 케일러 나바스, 페트르 체흐, 히카르두 오르타]"


# N그램 만들기

In [115]:
def make_ngram(x,WINDOW_SIZE):
    data = []
    for sentence in x:
        for idx,word in enumerate(sentence):
            for neighbor in sentence[max(idx - WINDOW_SIZE ,0) : min( idx+ WINDOW_SIZE, len(sentence))]:
                if neighbor != word:
                    data.append([word,neighbor])
    return data

In [116]:
data=make_ngram(raw_data['new_list'],3)
df=pd.DataFrame(data, columns = ['input','label'])

In [117]:
df['input'] = df['input'].map(word2int) 
df['label'] = df['label'].map(word2int) 
df['input'] = df['input'].apply(lambda x: int(x))
df['label'] = df['label'].map(lambda x: int(x))

In [118]:
df

,input,label
0,730,99
1,730,12
2,99,730
3,99,12
4,99,1282
...,...,...
20965,463,979
20966,463,1031
20967,1031,210
20968,1031,979


In [119]:
subset = df[['input', 'label']]
pairs = [tuple(x) for x in subset.to_numpy()]

In [120]:
pairs[1000]

(503, 1282)

In [121]:
print([k for k,v in word2int.items() if v == pairs[1][0]],
      [k for k,v in word2int.items() if v == pairs[1][1]])

['D. 마라도나'] ['리오넬 메시']


In [122]:
pairs_set = set(pairs)

---
# 모델 설계하기

In [123]:
import numpy as np
import random
random.seed(100)

def generate_batch(pairs, n_positive = 50, negative_ratio = 1.0, classification = False):
    """Generate batches of samples for training"""
    #배치사이즈
    batch_size = n_positive * (1 + negative_ratio)
    #배치사이즈 x 3 의 batch만들기
    batch = np.zeros((batch_size, 3))
    
    # Adjust label based on task
    if classification:
        neg_label = 0
    else:
        neg_label = -1
    
    # This creates a generator
    while True:
        # randomly choose positive examples 긍정라벨갯수만큼 뽑음
        for idx, (book_id, link_id) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (book_id, link_id, 1)

        # Increment idx by 1
        idx += 1
        
        # Add negative examples until reach batch size 부정라벨은 총 배치사이즈까지 뽑음
        while idx < batch_size:
            
            # random selection 임의로 뽑아서
            random_book = random.randrange(len(words))
            random_link = random.randrange(len(words))
            
            # Check to make sure this is not a positive example 페어셋에 있는지 확인하고
            if (random_book, random_link) not in pairs_set:
                
                # Add to batch and increment index 배치에 추가함
                batch[idx, :] = (random_book, random_link, neg_label)
                idx += 1
                
        # Make sure to shuffle order 랜덤 셔플합니다
        np.random.shuffle(batch)
        yield {'book': batch[:, 0], 'link': batch[:, 1]}, batch[:, 2]

In [124]:
def book_embedding_model(embedding_size = 50, classification = False):
    """Model to embed books and wikilinks using the functional API.
       Trained to discern if a link is present in a article"""
    
    # Both inputs are 1-dimensional
    book = Input(name = 'book', shape = [1])
    link = Input(name = 'link', shape = [1])
    
    # Embedding the book (shape will be (None, 1, 50))
    book_embedding =Embedding(name = 'book_embedding',
                                           input_dim = len(words),
                                           output_dim = embedding_size)(book)
    
    # Embedding the link (shape will be (None, 1, 50))
    link_embedding =Embedding(name = 'link_embedding',
                                                    input_dim = len(words),
                                                    output_dim = embedding_size)(link)
    
    # Merge the layers with a dot product along the second axis (shape will be (None, 1, 1))
    merged = Dot(name = 'dot_product', normalize = True, axes = 2)([book_embedding, link_embedding])
    
    # Reshape to be a single number (shape will be (None, 1))
    merged = Reshape(target_shape = [1])(merged)
    
    # If classifcation, add extra layer and loss function is binary cross entropy
    if classification:
        merged = Dense(1, activation = 'sigmoid')(merged)
        model = Model(inputs = [book, link], outputs = merged)
        model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    # Otherwise loss function is mean squared error
    else:
        model = Model(inputs = [book, link], outputs = merged)
        model.compile(optimizer = 'Adam', loss = 'mse')
    
    return model

# Instantiate model and show parameters
model = book_embedding_model()

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
book (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
link (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
book_embedding (Embedding)      (None, 1, 50)        74050       book[0][0]                       
__________________________________________________________________________________________________
link_embedding (Embedding)      (None, 1, 50)        74050       link[0][0]                       
__________________________________________________________________________________________________
dot_produc

In [125]:
n_positive =256

gen = generate_batch(pairs, n_positive, negative_ratio = 2)

# Train
h = model.fit_generator(gen, epochs = 50, 
                        steps_per_epoch = len(pairs) // n_positive,
                        verbose = 2 ,)

Epoch 1/50
 - 2s - loss: 0.9705
Epoch 2/50
 - 1s - loss: 0.8717
Epoch 3/50
 - 2s - loss: 0.7548
Epoch 4/50
 - 2s - loss: 0.5637
Epoch 5/50
 - 2s - loss: 0.3983
Epoch 6/50
 - 2s - loss: 0.3482
Epoch 7/50
 - 2s - loss: 0.3292
Epoch 8/50
 - 2s - loss: 0.3169
Epoch 9/50
 - 2s - loss: 0.3144
Epoch 10/50
 - 2s - loss: 0.3122
Epoch 11/50
 - 2s - loss: 0.3062
Epoch 12/50
 - 2s - loss: 0.3144
Epoch 13/50
 - 2s - loss: 0.3086
Epoch 14/50
 - 2s - loss: 0.3113
Epoch 15/50
 - 2s - loss: 0.3007
Epoch 16/50
 - 2s - loss: 0.3057
Epoch 17/50
 - 2s - loss: 0.3173
Epoch 18/50
 - 2s - loss: 0.3112
Epoch 19/50
 - 2s - loss: 0.3048
Epoch 20/50
 - 2s - loss: 0.3038
Epoch 21/50
 - 2s - loss: 0.3041
Epoch 22/50
 - 2s - loss: 0.2946
Epoch 23/50
 - 2s - loss: 0.3036
Epoch 24/50
 - 2s - loss: 0.3015
Epoch 25/50
 - 2s - loss: 0.3031
Epoch 26/50
 - 2s - loss: 0.3060
Epoch 27/50
 - 2s - loss: 0.2966
Epoch 28/50
 - 2s - loss: 0.3000
Epoch 29/50
 - 2s - loss: 0.3019
Epoch 30/50
 - 2s - loss: 0.2974
Epoch 31/50
 - 2s -

In [126]:
model.save('../first_attempt.h5')

In [127]:
# Extract embeddings
book_layer = model.get_layer('book_embedding')
book_weights = book_layer.get_weights()[0]
#book_weights = book_weights / np.linalg.norm(book_weights, axis = 1).reshape((-1, 1))

In [128]:
book_weights.shape

(1481, 50)

In [165]:
dists = np.dot(book_weights, book_weights[word2int.get('폴 포그바')])
dists

array([-0.02091598,  0.02145824,  0.00589058, ..., -0.04982658,
        0.05163328,  0.0429699 ], dtype=float32)

In [166]:
sorted_dists = np.argsort(dists)

In [167]:
n=10
closest = sorted_dists[-n-1: len(dists) - 1]

In [168]:
closest

array([ 485, 1366,  363,  883,   12,  436,  329,  827, 1370,  928],
      dtype=int64)

In [169]:
items=[]
for c in closest:
    a= [ k for k,v in word2int.items() if v == c ]
    items.append(a)

In [164]:
items

[['김태환'],
 ['고정운'],
 ['폴 스콜스'],
 ['로이 킨'],
 ['김민우'],
 ['이영재'],
 ['이재성'],
 ['기성용'],
 ['김도근'],
 ['박지성']]

In [ ]:
 distances = [dists[c] for c in closest]

In [ ]:
distances

[0.20154928,
 0.20335656,
 0.2040122,
 0.20891145,
 0.2112629,
 0.21163595,
 0.21341898,
 0.2137254,
 0.2161277,
 0.21939248]

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for key,value in word2int.items():
  vec = book_weights[value] # skip 0, it's padding.
  out_m.write(key + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [ ]:
for key,value in word2int.items():
  vec = book_weights[value] # skip 0, it's padding.
  #print(key + "\n")
  print('\t'.join([str(x) for x in vec]) + "\n")

-0.101256706	-0.04285067	-0.053068236	-0.03679687	-0.01269104	-0.014900399	0.06567214	0.060762357	-0.027672878	0.045187317	0.037859797	0.06228543	0.05485085	0.025041874	0.008204922	-0.081397	-0.09463666	-0.043842524	0.067428686	-0.00059446134	-0.040185183	0.032399453	-0.05894582	-0.05158283	-0.03964128	0.053714707	0.042223882	-0.07751818	-0.12113342	0.018851012	-0.08480692	0.06806589	-0.0067061153	0.07062124	0.023420725	-0.024158074	-0.06103691	-0.039723914	0.039290585	0.0711186	-0.09243434	-0.09037635	-0.050249457	0.06488526	-0.080418296	-0.019000344	0.054581	-0.08183745	0.047284294	0.09085079

0.023944909	0.01385373	0.0057541374	-0.03456691	0.094268255	-0.062306955	-0.012769426	0.008400483	0.08049839	0.034203418	-0.0031467099	0.033372674	-0.104895264	0.051691536	0.01969878	0.06343769	-0.087023	-0.05931328	-0.037294965	0.0069146235	0.07670387	0.0019219277	-0.04768909	-0.01579132	0.051508818	0.10950765	0.08511915	0.095674045	0.03648669	0.030072775	0.060252536	-0.0156236645	-0.010135427